In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import boxcox
pd.options.display.max_rows = 100
## Install xlrd package to load Excel files
#!conda install openpyxl
#!conda install xlrd

In [6]:
file1 = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv')

In [7]:
file1

,Customer,ST,GENDER,Education,Customer Lifetime Value,Income,Monthly Premium Auto,Number of Open Complaints,Policy Type,Vehicle Class,Total Claim Amount
0,RB50392,Washington,NaN,Master,NaN,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934
1,QZ44356,Arizona,F,Bachelor,697953.59%,0.0,94.0,1/0/00,Personal Auto,Four-Door Car,1131.464935
2,AI49188,Nevada,F,Bachelor,1288743.17%,48767.0,108.0,1/0/00,Personal Auto,Two-Door Car,566.472247
3,WW63253,California,M,Bachelor,764586.18%,0.0,106.0,1/0/00,Corporate Auto,SUV,529.881344
4,GA49547,Washington,M,High School or Below,536307.65%,36357.0,68.0,1/0/00,Personal Auto,Four-Door Car,17.269323
...,...,...,...,...,...,...,...,...,...,...,...
4003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Exercise 1: Cleaning Column Names
To ensure consistency and ease of use, standardize the column names of the dataframe. Start by taking a first look at the dataframe and identifying any column names that need to be modified. Use appropriate naming conventions and make sure that column names are descriptive and informative.

In [8]:
def lower_case_column_names(file1):
    file1.columns=[i.lower() for i in file1.columns]
    return file1

In [9]:
file1=lower_case_column_names(file1)

In [10]:
def rename_columns(file1):
    file1.rename(columns={'customer':'id','st':'location',
                          'customer lifetime value':'lifetime_value',"income":"income","monthly premium auto": "premium", 
                          "number of open complaints":"open_complaints","policy type": "policy",
                          "vehicle class":"vehicle", "total claim amount": "claim_amount"}, inplace=True )
    return file1


In [11]:
file1 = rename_columns(file1)
file1

,id,location,gender,education,lifetime_value,income,premium,open_complaints,policy,vehicle,claim_amount
0,RB50392,Washington,NaN,Master,NaN,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934
1,QZ44356,Arizona,F,Bachelor,697953.59%,0.0,94.0,1/0/00,Personal Auto,Four-Door Car,1131.464935
2,AI49188,Nevada,F,Bachelor,1288743.17%,48767.0,108.0,1/0/00,Personal Auto,Two-Door Car,566.472247
3,WW63253,California,M,Bachelor,764586.18%,0.0,106.0,1/0/00,Corporate Auto,SUV,529.881344
4,GA49547,Washington,M,High School or Below,536307.65%,36357.0,68.0,1/0/00,Personal Auto,Four-Door Car,17.269323
...,...,...,...,...,...,...,...,...,...,...,...
4003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Exercise 2: Cleaning invalid Values
The dataset contains columns with inconsistent and incorrect values that could affect the accuracy of our analysis. Therefore, we need to clean these columns to ensure that they contain only valid data. For example, the gender column contains various inconsistent values such as "F", "M", "Femal", "Male", "female", which need to be standardized.

Note that this exercise will focus only on cleaning inconsistent values and will not involve handling null values (NaN or None).

In [12]:
file1.gender.unique()

array([nan, 'F', 'M', 'Femal', 'Male', 'female'], dtype=object)

In [13]:
def clean_gender(x):
    if x in ['M', 'Male']:
        return 'Male'
    elif x in ['F', 'Femal',"female"]:
        return 'Female'
    else:
        return np.nan

In [14]:
file1['gender'] = list(map(clean_gender, file1['gender'])) 

In [15]:
file1.gender.value_counts()

Female    502
Male      452
Name: gender, dtype: int64

In [16]:
file1.location.unique()

array(['Washington', 'Arizona', 'Nevada', 'California', 'Oregon', 'Cali',
       'AZ', 'WA', nan], dtype=object)

In [17]:
def clean_location(x):
    if x in ['Washington', 'WA']:
        return 'Washington'
    elif x in ['Oregon']:
        return 'Oregon'
    elif x in ['Arizona', 'AZ']:
        return 'Arizona'
    elif x in ['Cali', 'California']:
        return 'California'
    elif x in ['Nevada']:
        return 'Nevada'
    else:
        return np.nan

In [18]:
file1['location'] = list(map(clean_location, file1['location'])) 
file1.location.value_counts()

California    331
Oregon        320
Arizona       211
Washington    111
Nevada         98
Name: location, dtype: int64

In [19]:
file1['location'] = file1['location'].astype(str)


In [20]:
file1.lifetime_value.info()

<class 'pandas.core.series.Series'>
RangeIndex: 4008 entries, 0 to 4007
Series name: lifetime_value
Non-Null Count  Dtype 
--------------  ----- 
1068 non-null   object
dtypes: object(1)
memory usage: 31.4+ KB


In [21]:
file1['lifetime_value'] = file1['lifetime_value'].str.replace('%', '').astype(float)



Exercise 3: Formatting data types
The data types of many columns in the dataset appear to be incorrect. This could impact the accuracy of our analysis. To ensure accurate analysis, we need to correct the data types of these columns. Please update the data types of the columns as appropriate.

It is important to note that this exercise does not involve handling null values (NaN or None).

In [22]:
column_mapping = {
    'id': str,
    'location': str,
    'lifetime_value': float,
    'income': float,
    'premium': float,
    'policy': str,
    'vehicle': str,
    'claim_amount': float
}



In [23]:
file1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4008 entries, 0 to 4007
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               1071 non-null   object 
 1   location         4008 non-null   object 
 2   gender           954 non-null    object 
 3   education        1071 non-null   object 
 4   lifetime_value   1068 non-null   float64
 5   income           1071 non-null   float64
 6   premium          1071 non-null   float64
 7   open_complaints  1071 non-null   object 
 8   policy           1071 non-null   object 
 9   vehicle          1071 non-null   object 
 10  claim_amount     1071 non-null   float64
dtypes: float64(4), object(7)
memory usage: 344.6+ KB


In [24]:
nan_percentage = file1.isnull().mean() * 100
nan_percentage


id                 73.278443
location            0.000000
gender             76.197605
education          73.278443
lifetime_value     73.353293
income             73.278443
premium            73.278443
open_complaints    73.278443
policy             73.278443
vehicle            73.278443
claim_amount       73.278443
dtype: float64

Exercise 4: Dealing with Null values
Identify any columns with null or missing values. Identify how many null values each column has. You can use the isnull() function in pandas to find columns with null values.

Decide on a strategy for handling the null values. There are several options, including:

Drop the rows or columns with null values
Fill the null values with a specific value (such as the column mean or median for numerical variables and mode for categorical variables)
Fill the null values with the previous or next value in the column
Fill the null values based on a more complex algorithm or model (we haven't studied this yet)
Implement your chosen strategy to handle the null values. You can use the fillna() function in pandas to fill null values or dropna() function to drop null values.

Verify that your strategy has successfully handled the null values. You can use the isnull() function again to check if there are still null values in the dataset.

Remember to document your process and explain your reasoning for choosing a particular strategy for handling null values.

After formatting data types, as a last step, convert all the numeric variables to integers using applymap.

Save the cleaned dataset to a new CSV file.

In [25]:
null_counts = file1.isnull().sum()
print(null_counts)


id                 2937
location              0
gender             3054
education          2937
lifetime_value     2940
income             2937
premium            2937
open_complaints    2937
policy             2937
vehicle            2937
claim_amount       2937
dtype: int64


In [26]:
threshold = 8
columns_to_check = ['id', 'location', 'gender', 'education', 'lifetime_value', 'income', 'premium', 'open_complaints', 'policy', 'vehicle','claim_amount'] 
file1_dropped = file1.dropna(subset=columns_to_check, thresh=threshold)



In [27]:
file1_dropped

,id,location,gender,education,lifetime_value,income,premium,open_complaints,policy,vehicle,claim_amount
0,RB50392,Washington,NaN,Master,NaN,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934
1,QZ44356,Arizona,Female,Bachelor,697953.59,0.0,94.0,1/0/00,Personal Auto,Four-Door Car,1131.464935
2,AI49188,Nevada,Female,Bachelor,1288743.17,48767.0,108.0,1/0/00,Personal Auto,Two-Door Car,566.472247
3,WW63253,California,Male,Bachelor,764586.18,0.0,106.0,1/0/00,Corporate Auto,SUV,529.881344
4,GA49547,Washington,Male,High School or Below,536307.65,36357.0,68.0,1/0/00,Personal Auto,Four-Door Car,17.269323
...,...,...,...,...,...,...,...,...,...,...,...
1066,TM65736,Oregon,Male,Master,305955.03,38644.0,78.0,1/1/00,Personal Auto,Four-Door Car,361.455219
1067,VJ51327,California,Female,High School or Below,2031499.76,63209.0,102.0,1/2/00,Personal Auto,SUV,207.320041
1068,GS98873,Arizona,Female,Bachelor,323912.47,16061.0,88.0,1/0/00,Personal Auto,Four-Door Car,633.600000
1069,CW49887,California,Female,Master,462680.11,79487.0,114.0,1/0/00,Special Auto,SUV,547.200000


In [28]:
nan_percentage = file1_dropped.isnull().mean() * 100
nan_percentage

id                  0.000000
location            0.000000
gender             10.924370
education           0.000000
lifetime_value      0.280112
income              0.000000
premium             0.000000
open_complaints     0.000000
policy              0.000000
vehicle             0.000000
claim_amount        0.000000
dtype: float64

In [29]:
file1_dropped = file1_dropped.reset_index(drop=True)




In [30]:
print(file1_dropped)

           id    location  gender             education  lifetime_value  \
0     RB50392  Washington     NaN                Master             NaN   
1     QZ44356     Arizona  Female              Bachelor       697953.59   
2     AI49188      Nevada  Female              Bachelor      1288743.17   
3     WW63253  California    Male              Bachelor       764586.18   
4     GA49547  Washington    Male  High School or Below       536307.65   
...       ...         ...     ...                   ...             ...   
1066  TM65736      Oregon    Male                Master       305955.03   
1067  VJ51327  California  Female  High School or Below      2031499.76   
1068  GS98873     Arizona  Female              Bachelor       323912.47   
1069  CW49887  California  Female                Master       462680.11   
1070  MY31220  California  Female               College       899704.02   

       income  premium open_complaints          policy        vehicle  \
0         0.0   1000.0    